앞서 배운 앞서 배운 피드 포워드 신경망은 입력의 길이가 고정되어 있어 자연어 처리를 위한 신경망으로는 한계가 있었음  
$\Longrightarrow$ 다양한 길이의 입력 시퀀스를 처리할 수 있는 인공 신경망이 필요해졌음

# 1. 순환 신경망(Recurrent Neural Network, RNN)
> 입력과 출력을 시퀀스 단위로 처리하는 모델

## 1) 순환 신경망(Recurrent Neural Network, RNN)
- 앞서 배운 Feed Forward NN처럼 출력층 방향으로만 결과를 보내는 것 이외에도 다른 신경망이 있음
- 그 중 하나가 RNN.
- RNN은 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징을 갖고있음

<img src=rnn_image1_ver2.png width=200>

- $x$ : 입력층의 입력벡터, $y$ : 출력층의 출력벡터
- 메모리 셀(cell) : RNN에서 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드
    - 이전의 값을 기억하려고 하는 일종의 메모리 역할을 수행함
- 각각의 시점(time step)에서 바로 이전 시점에서의 은닉층의 메모리 셀에서 나온 값을 자신의 입력으로 사용하는 재귀적 활동을 함
- 현재 시점 t에서의 메모리 셀이 갖고있는 값은 과거의 메모리 셀들의 값에 영향을 받음
- 은닉 상태(hidden state) : 메모리 셀이 출력층 방향으로 또는 다음 시점 t+1의 자신에게 보내는 값

<img src=rnn_image2_ver3.png width=300>

- 일반적으로 위의 그림에서 좌측과 같이 화살표로 사이클을 그려서 재귀 형태로 표현하거나, 화살표 대신 여러 시점으로 펼쳐서 표현하기도 함

<img src=rnn_image3_ver2.png width=300>

- 입력과 출력의 길이를 위와 같이 다르게 설계할 수 있음

- Examples
    - 하나의 입력에 대해서 여러개의 출력(1-to-many) 모델은 하나의 이미지 입력에 대해서 사진의 제목을 출력하는 이미지 캡셔닝(Image Captioning) 작업에 사용될 수 있음
    - 단어 시퀀스를 입력하고 하나의 출력(many-to-1) 모델은 입력문서가 긍정적인지 부정적인지를 판별하는 감성 분류(sentiment classification), 스펨메일 판별(spam detection)에 사용 될 수 있음
    
<img src=rnn_image3.5.png width=200>

- Examples
    - 입력 문장으로부터 대답 문장을 출력하는 챗봇과 번역기, ch12에서 배우는 개체명 인식이나 품사 태깅과 같은 작업
  

<img src=rnn_image3.7.png width=200>

### RNN 수식

<img src=rnn_image4_ver2.png width=100>

- $h_t$ : 시점 t에서의 은닉 상태값
- 은닉층 : $h_t = tanh(W_xx_t + W_hh_{t-1} + b)$
- 출력층 : $y_t = f(W_yh_t + b)$, where $f$ : 비선형함수

---

<img src=rnn_images4-5.png width=500>

In NLP,
- $D_h$ : 은닉 상태의 크기, $d$ : 단어 벡터의 차원
- $x_t$ : 단어 벡터
- 활성화함수로 tanh가 주로 사용되지만 ReLU로 바꿔 사용하는 시도도 있음
- 출력층은 결과값인 $y_t$를 계산하기 위한 활성화함수를 선택할 때,
    - 이진 분류를 해야하는 경우 : 시그모이드함수
    - 다중 분류를 해야하는 경우 : 소프트맥스함수

## 2) 케라스(Keras)로 RNN 구현하기

In [3]:
from keras.models import Sequential
from keras.layers import SimpleRNN

### RNN 층을 추가하는 코드.
model.add(SimpleRNN(hidden_size))

### 추가 인자를 사용할 때
model.add(SimpleRNN(hidden_size, input_shape=(timesteps, input_dim)))
- hidden_size = 은닉 상태의 크기를 정의. 메모리 셀이 다음 시점의 메모리 셀과 출력층으로 보내는 값의 크기(output_dim)와도 동일. RNN의 용량(capacity)을 늘린다고 보면 되며, 중소형 모델의 경우 보통 128, 256, 512, 1024 등의 값을 가진다.
- timesteps = 입력 시퀀스의 길이(input_length)라고 표현하기도 함. 시점의 수.
- input_dim = 입력의 크기.


### 다른 표기
model.add(SimpleRNN(hidden_size, input_length=M, input_dim=N)) (**단, M과 N은 정수**)

<img src=rnn_image6between7.png width=400>

- RNN 층은 (batch_size, timesteps, input_dim) 크기의 3D 텐서를 입력으로 받음
    - batch_size는 한 번에 학습하는 데이터의 개수
- 위의 코드는 출력층까지 포함한 하나의 완성된 인공 신경망 코드가 아니라 은닉층. 즉, RNN 층에 대한 코드

RNN 층은 사용자의 설정에 따라 두 가지 종류의 출력을 내보냅니다.
- 메모리 셀의 최종 시점의 은닉 상태만을 리턴하고자 한다면 (batch_size, output_dim) 크기의 2D 텐서를 리턴합니다.
- 메모리 셀의 각 시점(time step)의 은닉 상태값들을 모아서 전체 시퀀스를 리턴하고자 한다면 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴합니다.
- 이는 RNN 층의 return_sequences 매개 변수에 True를 설정하여 설정이 가능합니다. (output_dim은 앞서 코드에서 정의한 hidden_size의 값으로 설정됩니다.)

<img src=rnn_image8_ver2.png width=300>

- time step=3일 때, return_sequences = True를 설정했을 때와 그렇지 않았을 때 어떤 차이가 있는지를 보여줌
    - return_sequences=True를 선택하면 메모리 셀이 모든 시점(time step)에 대해서 은닉 상태값을 출력하며, 
    - 별도 기재하지 않거나 return_sequences=False로 선택할 경우에는 메모리 셀은 하나의 은닉 상태값만을 출력합니다.
    

마지막 은닉 상태만 전달하도록 하면 many-to-one 문제를 풀 수 있고, 모든 시점의 은닉 상태를 전달하도록 하면, 다음층에 은닉층이 하나 더 있는 경우이거나 many-to-many 문제를 풀 수 있습니다.

뒤에서 배우는 LSTM이나 GRU도 내부 메커니즘은 다르지만 model.add()를 통해서 층을 추가하는 코드는 사실상 SimpleRNN 코드와 같은 형태를 가집니다. 실습을 통해 모델 내부적으로 출력 결과를 어떻게 정의하는지 보면서 RNN을 이해해보겠습니다.

In [2]:
from keras.models import Sequential
from keras.layers import SimpleRNN

Using TensorFlow backend.


In [3]:
model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일함.
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


### 위에서 parameter 구하는 것이 이해가 잘 안되서 가져온 예제
- model.add(SimpleRNN(4, input_shape=(2,3)) 
<img src=71An9.png width=500>

(number of parameters) = (num of units)$^2$ + (num of units * input dim) + (num of units)  
= **(num of units) * (num of units + input dim + 1)**

따라서, 여기서는 답이 $4 * (4+3+1) = 32$

In [4]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (8, 2, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


## 3) 파이썬으로 RNN 구현하기
> Numpy로 RNN층 구현할 예정

- 앞서 메모리 셀에서 은닉상태를 계산하는 식은 다음과 같았음
$h_t = tanh(W_xX_t+W_h+h_{t-1}+b)$

In [5]:
import numpy as np

timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_dim = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다.
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량이다.

inputs = np.random.random((timesteps, input_dim)) # 입력에 해당되는 2D 텐서

hidden_state_t = np.zeros((hidden_size,)) # 초기 은닉 상태는 0(벡터)로 초기화
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬.

print(hidden_state_t) # 8의 크기를 가지는 은닉 상태. 현재는 초기 은닉 상태로 모든 차원이 0의 값을 가짐.

[0. 0. 0. 0. 0. 0. 0. 0.]
